In [32]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [33]:
df = pd.read_csv('./data/raw/vgsales.csv')
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [34]:
df.describe()

,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,16598.000000,16327.000000,16598.000000,16598.000000,16598.000000,16598.000000,16598.000000
mean,8300.605254,2006.406443,0.264667,0.146652,0.077782,0.048063,0.537441
std,4791.853933,5.828981,0.816683,0.505351,0.309291,0.188588,1.555028
min,1.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.010000
25%,4151.250000,2003.000000,0.000000,0.000000,0.000000,0.000000,0.060000
50%,8300.500000,2007.000000,0.080000,0.020000,0.000000,0.010000,0.170000
75%,12449.750000,2010.000000,0.240000,0.110000,0.040000,0.040000,0.470000
max,16600.000000,2020.000000,41.490000,29.020000,10.220000,10.570000,82.740000


In [35]:
df.isnull().sum()

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

In [36]:
df['sales_vs_genre_avg'] = df.groupby('Genre')['Global_Sales'].transform('mean').round(2)
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,sales_vs_genre_avg
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,0.57
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,0.94
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,0.59
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,0.57
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,0.62


In [37]:
df['above_genre_avg'] = df.Global_Sales > df.sales_vs_genre_avg
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,sales_vs_genre_avg,above_genre_avg
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,0.57,True
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,0.94,True
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,0.59,True
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,0.57,True
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,0.62,True


In [38]:
#Rank games within each genre
df['genre_rank'] = df.groupby('Genre')['Global_Sales'].rank(method='dense', ascending=False)
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,sales_vs_genre_avg,above_genre_avg,genre_rank
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,0.57,True,1.0
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,0.94,True,1.0
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,0.59,True,1.0
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,0.57,True,2.0
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,0.62,True,1.0


In [39]:
df['top_3_in genre'] = df['genre_rank'] <= 3
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,sales_vs_genre_avg,above_genre_avg,genre_rank,top_3_in genre
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,0.57,True,1.0,True
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,0.94,True,1.0,True
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,0.59,True,1.0,True
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,0.57,True,2.0,True
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,0.62,True,1.0,True


In [40]:
df['percentile_within_genre'] = df.groupby('Genre')['Global_Sales'].rank(pct=True)
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,sales_vs_genre_avg,above_genre_avg,genre_rank,top_3_in genre,percentile_within_genre
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,0.57,True,1.0,True,1.000000
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,0.94,True,1.0,True,1.000000
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,0.59,True,1.0,True,1.000000
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,0.57,True,2.0,True,0.999574
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,0.62,True,1.0,True,1.000000


In [41]:
platform_pub_sales = df.groupby(['Platform', 'Publisher'])['Global_Sales'].agg('sum').reset_index()
platform_pub_sales

,Platform,Publisher,Global_Sales
0,2600,20th Century Fox Video Games,1.94
1,2600,Activision,19.42
2,2600,Answer Software,0.50
3,2600,Atari,50.09
4,2600,Avalon Interactive,0.17
...,...,...,...
1796,XOne,Telltale Games,0.46
1797,XOne,Tru Blu Entertainment,0.05
1798,XOne,Ubisoft,17.91
1799,XOne,Unknown,0.03


In [46]:
max_publisher = platform_pub_sales.loc[platform_pub_sales.groupby('Platform')['Global_Sales'].idxmax()]
dominant = max_publisher[['Platform', 'Publisher']]

In [44]:
df = df.merge(dominant, on='Platform', how='left', suffixes=('', '_dominant'))
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,sales_vs_genre_avg,above_genre_avg,genre_rank,top_3_in genre,percentile_within_genre,Publisher_dominant
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,0.57,True,1.0,True,1.000000,Nintendo
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,0.94,True,1.0,True,1.000000,Nintendo
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,0.59,True,1.0,True,1.000000,Nintendo
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,0.57,True,2.0,True,0.999574,Nintendo
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,0.62,True,1.0,True,1.000000,Nintendo


In [45]:
df['is_dominant_publisher'] = df['Publisher'] == df['Publisher_dominant']
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,sales_vs_genre_avg,above_genre_avg,genre_rank,top_3_in genre,percentile_within_genre,Publisher_dominant,is_dominant_publisher
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,0.57,True,1.0,True,1.000000,Nintendo,True
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,0.94,True,1.0,True,1.000000,Nintendo,True
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,0.59,True,1.0,True,1.000000,Nintendo,True
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,0.57,True,2.0,True,0.999574,Nintendo,True
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,0.62,True,1.0,True,1.000000,Nintendo,True


In [59]:
#Year each platform had highest total global_sales
platform_yearly_total = df.groupby(['Platform', 'Year'])['Global_Sales'].agg('sum').reset_index()
platform_max_year = platform_yearly_total.loc[platform_yearly_total.groupby('Platform')['Global_Sales'].idxmax()]
platform_max_year

,Platform,Year,Global_Sales
1,2600,1981.0,35.77
11,3DO,1995.0,0.08
12,3DS,2011.0,62.53
20,DC,2000.0,5.99
29,DS,2007.0,149.36
39,GB,1989.0,64.98
55,GBA,2004.0,78.09
60,GC,2002.0,51.93
68,GEN,1992.0,12.66
71,GG,1992.0,0.04
